In [5]:
from visualize.hindsight_improvability import get_predictions
from eval import get_preds_earlyexiting

import argparse
from pathlib import Path

import os

import torch

In [6]:
torch.cuda.is_available()

False

In [2]:
os.environ['CIFAR100_PATH'] = "/home/mjazbec/poe/Zero-Time-Waste/ztw_rewrite/data/cifar100"

In [3]:
parser = argparse.ArgumentParser()
ARGS = parser.parse_args(args=[])
ARGS.runs_dir = Path.cwd() / 'runs'
ARGS.exp_names = ["cifar100_resnet50_IHYWUAYB", "cifar100_ztw_cascading_5377D53A", "cifar100_ztw_ensembling_SOVJE4OA"]
ARGS.exp_ids = [0]
ARGS.display_names = None
ARGS.use_wandb = False
ARGS.output_dir = Path.cwd() / 'figures'
ARGS.output_name = 'hi'

In [7]:
for exp_name in ARGS.exp_names:
    for exp_id in ARGS.exp_ids:
        run_name = f'{exp_name}_{exp_id}'
        print(f'Calculating hindsight improvabilities for: {run_name}')
        ee_preds, labels = get_predictions(ARGS, run_name)
        if ee_preds is not None:
            print(exp_name)
            print(ee_preds.shape, labels.shape)

Calculating hindsight improvabilities for: cifar100_resnet50_IHYWUAYB_0
Calculating hindsight improvabilities for: cifar100_ztw_cascading_5377D53A_0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
